# NFL Baseline
- create target_df (distance in tracking_df is lower than threshold=3)
https://www.kaggle.com/code/stgkrtua/nfl-creatatraindataset-targetdf
- create dataset save frames in target_df
https://www.kaggle.com/code/stgkrtua/nfl-createdataset-saveframes
- check saved images
https://www.kaggle.com/code/stgkrtua/nfl-checkdataset-plotsavedimage

# import libraries

In [1]:
# general
import os
import gc
import pickle
import glob
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import cv2
import matplotlib.pyplot as plt
import time
import math

import sys
sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')
import timm


# deep learning
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD, Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import GroupKFold

# loss metrics
from sklearn.metrics import matthews_corrcoef, confusion_matrix, roc_auc_score

# import cudf
import polars as pl

import mlflow
# import wandb
# warningの表示方法の設定
import warnings
warnings.filterwarnings("ignore")

# Set Configurations

In [2]:
CFG = {
        "kaggle" : False,
        "DEBUG" : False,
        # model config
        "model_name" : "tf_efficientnet_b0",
        "out_features" : 20,
        "inp_channels": 3,
        "num_img_feature" : 5,
        "pretrained" : True,
        
        "roll_sum_window_size" : 10,
        "features" : ['x_position_1', 'y_position_1', 'x_position_2', 'y_position_2', 
                      'speed_1', 'distance_1', 'direction_1', 'orientation_1','acceleration_1', 'sa_1', 
                      'speed_2', 'distance_2', 'direction_2', 'orientation_2', 'acceleration_2', 'sa_2',
                      'players_dis', #'is_ground'
                    #   'speed_diff', 'distance_diff', 'direction_diff', 'orientation_diff','acceleration_diff', 'sa_diff', # diff-feature
                    #   'x_move_distance_1', 'y_move_distance_1', 'x_move_distance_2', 'y_move_distance_2',
                    #   'x_position_diff_rollsum','y_position_diff_rollsum', 
                    #   'speed_diff_rollsum','distance_diff_rollsum', 'direction_diff_rollsum','orientation_diff_rollsum', 
                    #   'acceleration_diff_rollsum', 'sa_diff_rollsum','players_dis_rollsum', 
                      ],

        # learning config
        "n_epoch" : 5,
        "n_folds": 3,
        "train_folds" : [0,1,2],
        "lr" : 1e-4,
        "T_max" : 10,
        "min_lr" : 1e-8,
        "weight_decay" : 1e-6,

        # etc
        "print_freq" : 1000,
        "random_seed" : 21,

        # data config    
        "img_size" : (224, 224),
        "batch_size" : 128,
        "num_workers" : 2,
        "masksize_helmet_ratio" : 4, # helmetサイズにこの係数をかけたサイズだけ色を残して後は黒塗りする
        "TRAIN_VIDEO_NUM" : 100,
        "VALID_VIDEO_NUM" : 10,
        "sample_num" : -1, 

        "EXP_CATEGORY" : "exps_polars",
        "EXP_NAME" : "expP005_PandG_bothview_notNormTenV2_effb0",
}

if CFG["DEBUG"]:
    CFG["EXP_CATEGORY"] = "DEBUG"
    CFG["EXP_NAME"] = "DEBUG"
    CFG["n_epoch"] = 2
    CFG["sample_num"] = 1000
    CFG["batch_size"] = 32
    CFG["train_folds"] : [0,1]


CFG["INPUT_DIR"] = "/workspace/input"
CFG["OUTPUT_DIR"] = "/workspace/output"
CFG["TRAIN_HELMET_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_baseline_helmets.csv")
CFG["TRAIN_TRACKING_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_player_tracking.csv")
CFG["TRAIN_VIDEO_META_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_video_metadata.csv")
CFG["TRAIN_LABEL_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_labels.csv")
CFG["SAVED_CONTACT_CSV"] = os.path.join(CFG["INPUT_DIR"], "Saved_contact_frames.csv")
CFG["CONTACT_IMG_DIR"] = os.path.join(CFG["INPUT_DIR"], "contact_images")
CFG["MODEL_DIR"] = os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"] ,"model")
    
if not CFG["kaggle"] and not CFG["DEBUG"]:
    os.mkdir(os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"]))
    os.mkdir(CFG["MODEL_DIR"])


In [3]:
if CFG["kaggle"]:
    os.environ["WANDB_SILENT"] = "true"
    WANDB_CONFIG = {'competition': 'NFL', '_wandb_kernel': 'taro'}
    # Secrets
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb")

    !wandb login $secret_value_0
    #! TODO : logger settings
    wandb.init(project=WANDB_CONFIG["competition"], config=CFG, group=CFG["EXP_CATEGORY"], name=CFG["EXP_NAME"])

else:
    mlflow.set_tracking_uri("/workspace/mlruns")
    experiment = mlflow.get_experiment_by_name(CFG["EXP_CATEGORY"])
    if experiment is None:  # 当該Experiment存在しないとき、新たに作成
        experiment_id = mlflow.create_experiment(name=CFG["EXP_CATEGORY"])
    else: # 当該Experiment存在するとき、IDを取得
        experiment_id = experiment.experiment_id

# Utils

In [4]:
def logging_metrics_epoch(fold, epoch, train_loss_avg, valid_loss_avg, score, threshold, tn_best, fp_best, fn_best, tp_best, auc_score):
    if CFG["kaggle"]:
        wandb.log({"loss avg":{f"train/fold{fold}": train_loss_avg,
                                f"valid/fold{fold}": valid_loss_avg}}, step=epoch)
        wandb.log({"Metircs" : {f"score/fold{fold}":score,
                                f"score threshold/fold{fold}":threshold,
                                f"tn/fold{fold}":tn_best,
                                f"fp/fold{fold}":fp_best,
                                f"fn/fold{fold}":fn_best,
                                f"tp/fold{fold}":tp_best,
                                f"auc/fold{fold}":auc_score,
                               }}, step=epoch)
    else:
        mlflow.log_metric(f"fold{fold} train loss avg", train_loss_avg, step=epoch)
        mlflow.log_metric(f"fold{fold} valid loss avg", valid_loss_avg, step=epoch)
        mlflow.log_metric(f"fold{fold} score", score, step=epoch)
        mlflow.log_metric(f"fold{fold} score threshold", threshold, step=epoch)
        mlflow.log_metric(f"fold{fold} tn", tn_best, step=epoch)
        mlflow.log_metric(f"fold{fold} fp", fp_best, step=epoch)
        mlflow.log_metric(f"fold{fold} fn", fn_best, step=epoch)
        mlflow.log_metric(f"fold{fold} tp", tp_best, step=epoch)
        mlflow.log_metric(f"fold{fold} auc", auc_score, step=epoch)

In [5]:
def seed_everything(seed=CFG["random_seed"]):
    #os.environ['PYTHONSEED'] = str(seed)
    np.random.seed(seed%(2**32-1))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark = False
seed_everything()

# device optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Using device: {device}')

Using device: cuda


In [6]:
def asMinutes(s):
    """Convert Seconds to Minutes."""
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    """Accessing and Converting Time Data."""
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

class AverageMeter(object):
    """Computes and stores the average and current value."""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# Dataset Functions

In [7]:
def add_feature_cols(df_, FEATURE_COLS,remove_col_list):
    additional_cols = list(df_.columns)
    additional_cols = [col for col in additional_cols if not col in remove_col_list]
    FEATURE_COLS.extend(additional_cols)
    return FEATURE_COLS

## target df func

In [8]:
# -------------------------------
# replace polars
# -------------------------------
def create_trackmerged_ftr(target_df, FEATURE_COLS,
                           diff_cols = ['x_position', 'y_position', 'speed', 'distance',
                                        'direction', 'orientation', 'acceleration', 'sa']):
    
    
    target_df = target_df.with_column((np.sqrt(np.square(pl.col("x_position_1")-pl.col("x_position_2")) \
                                              + np.square(pl.col("y_position_1")-pl.col("y_position_2"))) \
                                      ).alias("players_dis"))
    target_df = target_df.with_column(pl.col("players_dis").fill_null(0))
    
    # players distance sum(in shift range time : default(-6~6 frames not step))
    players_distance_sum = 0
    for idx in range(-6,6,1):
        players_distance_sum += np.sqrt((target_df[f"x_position_shift{idx}_1"] - target_df[f"x_position_shift{idx}_2"])**2 \
                                       + (target_df[f"y_position_shift{idx}_1"] - target_df[f"y_position_shift{idx}_2"])**2)
    target_df = target_df.with_column(pl.Series("players_distance_sum", players_distance_sum))
    target_df = target_df.with_column(pl.col("players_distance_sum").fill_null(0))
    FEATURE_COLS.append("players_distance_sum")

    # players each axis distance sum(in shift range time : default(-6~6 frames not step))
    for axis in ["x", "y"]:
        axis_distance_1 = 0
        axis_distance_2 = 0
        for idx in range(-6, 5, 1):
            axis_distance_1 += abs(target_df[f"{axis}_position_shift{idx}_1"] - target_df[f"{axis}_position_shift{idx+1}_1"])
            axis_distance_2 += abs(target_df[f"{axis}_position_shift{idx}_2"] - target_df[f"{axis}_position_shift{idx+1}_2"])
        target_df = target_df.with_column(pl.Series(f"{axis}_move_distance_1", axis_distance_1))
        target_df = target_df.with_column(pl.col(f"{axis}_move_distance_1").fill_null(0))
        target_df = target_df.with_column(pl.Series(f"{axis}_move_distance_2", axis_distance_2))
        target_df = target_df.with_column(pl.col(f"{axis}_move_distance_2").fill_null(0))
        FEATURE_COLS.extend([f"{axis}_move_distance_1", f"{axis}_move_distance_2"])

    # players difference ftr (in each step)
    for col in diff_cols:
        colname = f"{col}_diff"
        target_df = target_df.with_column((abs(pl.col(f"{col}_1") - pl.col(f"{col}_2"))).alias(colname))
        target_df = target_df.with_column(pl.col(colname).fill_null(0))
        FEATURE_COLS.append(colname)
        
    return target_df, FEATURE_COLS

In [9]:
# -------------------------------
# replace polars
# polars groupby rolling (groupby_dynamics)
# https://pola-rs.github.io/polars/py-polars/html/reference/dataframe/api/polars.DataFrame.groupby_dynamic.html
# -------------------------------
def create_roll_ftr(target_df, FEATURE_COLS,
                    key_cols = ["contact_id", "game_play", "nfl_player_id_1", 
                                "nfl_player_id_2", "datetime", "step"]):
    roll_df = target_df.select(roll_cols+key_cols)

    roll_df = roll_df.with_column(pl.concat_str([pl.col("game_play"),
                                                 pl.col("nfl_player_id_1"),
                                                 pl.col("nfl_player_id_2"),
                                                ], sep='_').alias('key'))
    # cast datetime
    roll_df = roll_df.with_column(pl.col('datetime').str.strptime(pl.Datetime,
                                                                          fmt="%+",
                                                                          strict=False
                                                                         ).alias('datetime'))
    # groupby rolling
    roll_df = roll_df.groupby_dynamic("step", every="1i", period="6i", by="key", closed="both").agg([pl.col(roll_cols).sum().suffix("_rollsum"), pl.col("contact_id")])
    roll_df = roll_df.with_column(pl.col("contact_id").apply(lambda x:x[0]))
    roll_df = roll_df.drop(["key", "step"])
    target_df = target_df.join(roll_df, on="contact_id", how="left")
    additional_cols = [col+"_rollsum" for col in roll_cols]
    FEATURE_COLS.extend(additional_cols)
    return target_df, FEATURE_COLS

In [10]:
# -------------------------------
# replace polars
# -------------------------------
def create_helmetmerged_ftr(target_df, FEATURE_COLS):
    # helmet center distance feature
    for view in ["Endzone", "Sideline"]:
        for p_id in ["1", "2"]: 
            # get helmet center
            target_df = target_df.with_column((pl.col(f"{view[0]}_left_{p_id}") + (pl.col(f"{view[0]}_width_{p_id}")//2)).alias(f"{view[0]}_Wcenter_{p_id}"))
            target_df = target_df.with_column((pl.col(f"{view[0]}_top_{p_id}") + (pl.col(f"{view[0]}_height_{p_id}")//2)).alias(f"{view[0]}_Hcenter_{p_id}"))
        # helmet center distance
        target_df = target_df.with_column((np.sqrt(np.square(pl.col(f"{view[0]}_Wcenter_1") - pl.col(f"{view[0]}_Wcenter_2")) \
                                                  + np.square(pl.col(f"{view[0]}_Hcenter_1") - pl.col(f"{view[0]}_Hcenter_2")))
                                          ).alias(f"{view[0]}_helmet_dis"))

        # GがNanになるので0にしておく
        target_df = target_df.with_column(pl.col(f"{view[0]}_helmet_dis").fill_null(0))
        FEATURE_COLS.append(f"{view[0]}_helmet_dis")
    
    # helmet cols fillna(0) after get helmet distance 
    helmet_cols = ['E_left_1', 'E_width_1', 'E_top_1', 'E_height_1',
                   'E_left_2', 'E_width_2', 'E_top_2', 'E_height_2', 
                   'S_left_1','S_width_1', 'S_top_1', 'S_height_1', 
                   'S_left_2', 'S_width_2', 'S_top_2', 'S_height_2']
    target_df = target_df.with_column(pl.col(helmet_cols).fill_null(0))

    return target_df, FEATURE_COLS

In [11]:
# -------------------------------
# replace polars
# -------------------------------
def get_categorical_ftr(target_df, FEATURE_COLS):
    target_df = target_df.with_column((pl.col("nfl_player_id_2")== "G").cast(int).alias("is_ground"))
    
#     target_df["nfl_player_id_2"] = target_df["nfl_player_id_2"].replace("G", "99999").astype(np.int64) # when inference this is after cnn pred
    target_df = target_df.with_column((1 - ((pl.col("E_width_1")==0) & (pl.col("E_width_2")==0) \
                                          & (pl.col("S_width_1")==0) & (pl.col("S_width_2")==0)).cast(int)).alias("is_helmet"))
    target_df = target_df.with_column((1 - ((pl.col("E_width_1")==0) & (pl.col("E_width_2")==0)).cast(int)).alias("is_E_helmet"))
    target_df = target_df.with_column((1 - ((pl.col("S_width_1")==0) & (pl.col("S_width_2")==0)).cast(int)).alias("is_S_helmet"))
    target_df = target_df.with_column(((pl.col("is_E_helmet")==1) & (pl.col("is_S_helmet")==1)).cast(int).alias("both_helmet"))
    # set team 
    target_df = target_df.with_column(((pl.col("team_1")=="home").cast(int)).alias("team_1"))
    target_df = target_df.with_column(((pl.col("team_2")=="home").cast(int)).alias("team_2"))
    
    target_df = target_df.fill_null(0)
    target_df = target_df.fill_nan(0)
    FEATURE_COLS.extend(["is_ground", "is_helmet"])
    return target_df, FEATURE_COLS

## tracking df func

In [12]:
# -------------------------------
# replace polars
# -------------------------------
def get_tracking_shift(tracking_df, shift_cols=["x_position","y_position"], shift_nums=range(-6,6,1)):
    # get shift key
#     tracking_df = tracking_df.with_column((pl.col('step')/10*59.94+5*59.94 + 5000).alias('frame_add'))
    tracking_df = tracking_df.with_column(pl.concat_str([pl.col("game_play"),
                                                         pl.col("nfl_player_id"),
                                                        ], sep='_').alias('shift_key'))
    # get shift features
    SHIFT_COLS = []
    for num in shift_nums:
        tracking_df = tracking_df.with_columns(
                            pl.col(shift_cols).shift(periods=num).over("shift_key").suffix(f"_shift{num}"))
    for col in shift_cols:
        colname = [f"{col}_shift{idx}" for idx in shift_nums]
        SHIFT_COLS.extend(colname)
                    
    return tracking_df, SHIFT_COLS

In [13]:
# -------------------------------
# replace polars
# -------------------------------
def target_merge_tracking(target_df, tracking_df, FEATURE_COLS, SHIFT_COLS,
                          TRACKING_COLS = ["game_play", "nfl_player_id", "step", 
                                           "x_position", "y_position", "datetime",
                                           "speed","distance","direction","orientation",
                                           "acceleration","sa", "team", "jersey_number"]):
    target_df = target_df.with_column(pl.concat_str([pl.col("game_play"),
                                                     pl.col("step").cast(str),
                                                     pl.col("nfl_player_id_1"),
                                                    ], sep='_').alias('game_step_player_1'))
    target_df = target_df.with_column(pl.concat_str([pl.col("game_play"),
                                                     pl.col("step").cast(str),
                                                     pl.col("nfl_player_id_2"),
                                                    ], sep='_').alias('game_step_player_2'))

    TRACKING_COLS.extend(SHIFT_COLS)
    tracking_df = tracking_df.select(TRACKING_COLS)
    tracking_df = tracking_df.with_column(pl.concat_str([pl.col("game_play"),
                                                         pl.col("step").cast(str),
                                                         pl.col("nfl_player_id"),
                                                        ], sep='_').alias('game_step_player'))

    tracking_df = tracking_df.drop(["game_play", "step", "nfl_player_id", "datetime"])

    # merge tracking to target
    for player_id in [1,2]:
        tracking_player = tracking_df.select([pl.all().suffix(f"_{player_id}")])
        target_df = target_df.join(tracking_player, on=[f"game_step_player_{player_id}"], how="left")
        # add features col
        FEATURE_COLS = add_feature_cols(tracking_player, FEATURE_COLS,
                                        [f"game_step_player_{player_id}", f"frame_{player_id}", f"datetime_{player_id}"])
    # drop col
    target_df = target_df.drop(["game_step_player_1", "game_step_player_2"])
    print(len(target_df.columns))
    print("original length", len(target_df))
    return target_df, FEATURE_COLS

## helmet df func

In [14]:
# -------------------------------
# replace polars
# -------------------------------
def target_merge_helmet(target_df, helmet_df, FEATURE_COLS):
    print("original length", len(target_df))
    # set merge-key (game_frame_player_1,2) to merge helmet_df
    target_df = target_df.with_column(((pl.col("step")/10*59.94+5*59.94).cast(int)+1).alias("frame"))
    target_df = target_df.with_column(pl.concat_str([pl.col("game_play"),
                                                     pl.col("frame").cast(str),
                                                     pl.col("nfl_player_id_1"),
                                                    ], sep='_').alias('game_frame_player_1'))
    target_df = target_df.with_column(pl.concat_str([pl.col("game_play"),
                                                     pl.col("frame").cast(str),
                                                     pl.col("nfl_player_id_2"),
                                                    ], sep='_').alias('game_frame_player_2'))
    # set merge key
    helmet_df = helmet_df.with_column(pl.concat_str([pl.col("game_play"),
                                                     pl.col("frame").cast(str),
                                                     pl.col("nfl_player_id"),
                                                    ], sep='_').alias('game_frame_player'))

    # merge target df & helmet_df
    player_views = [[1, "Endzone"],[2, "Endzone"], [1, "Sideline"],[2, "Sideline"]]
    for player_id, view in player_views:
        helmet_view = helmet_df.filter(pl.col("view")==view)
        helmet_view = helmet_view[["game_frame_player", "left", "width", "top", "height"]]
        helmet_view = helmet_view.select(pl.all().suffix(f"_{player_id}"))
        helmet_view = helmet_view.select([pl.col(helmet_view.columns[0]), pl.col(helmet_view.columns[1:]).prefix(f"{view[0]}_")])
        target_df = target_df.join(helmet_view, on=f"game_frame_player_{player_id}", how="left")  
        # add features col
        FEATURE_COLS = add_feature_cols(helmet_view, FEATURE_COLS, [f"game_frame_player_{player_id}"])

    print(len(target_df.columns))
    print("original length", len(target_df))
    return target_df, FEATURE_COLS

---

# Load Target

In [15]:
target_dtypes = {'contact_id':str, 
                'game_play':str,
                'datetime':str,
                'step':int,
                'nfl_player_id_1':str,
                'nfl_player_id_2':str,
                'contact':int,
                }
target_df = pl.read_csv(CFG["TRAIN_LABEL_CSV"], dtypes=target_dtypes)    

FEATURE_COLS = ["nfl_player_id_1", "nfl_player_id_2", "step"]
# display(target_df)

# Merge tracking_df

In [16]:
# -------------------------------
# replace polars
# -------------------------------
tracking_df = pl.read_csv(CFG["TRAIN_TRACKING_CSV"])
tracking_df, SHIFT_COLS = get_tracking_shift(tracking_df)
target_df, FEATURE_COLS = target_merge_tracking(target_df, tracking_df, FEATURE_COLS, SHIFT_COLS)
# display(tracking_df.filter((pl.col("game_play")=="58580_001136") & (pl.col("nfl_player_id").cast(str)=="44830")))# ちゃんとshiftできてそう
del tracking_df

target_df, FEATURE_COLS = create_trackmerged_ftr(target_df, FEATURE_COLS)

75
original length 4721618


# Exclude distance 2

In [17]:
print(len(target_df))
target_df = target_df.filter(pl.col("players_dis") <= 2)
print(len(target_df))

4721618
660553


# Merge helmet df

In [18]:
helmet_df = pl.read_csv(CFG["TRAIN_HELMET_CSV"])
target_df, FEATURE_COLS = target_merge_helmet(target_df, helmet_df, FEATURE_COLS)
target_df, FEATURE_COLS = create_helmetmerged_ftr(target_df, FEATURE_COLS)
target_df, FEATURE_COLS = get_categorical_ftr(target_df, FEATURE_COLS)
print(len(target_df))

original length 660553
108
original length 660553
660553


# Reduce Data

In [19]:
target_df = target_df.to_pandas()
target_df_E = target_df[target_df["is_E_helmet"]==1]
target_df_E["EorS"] = "Endzone"

target_df_S = target_df[target_df["is_S_helmet"]==1]
target_df_S["EorS"] = "Sideline"

target_df = pd.concat([target_df_E, target_df_S], axis=0).reset_index(drop=True)

if CFG["DEBUG"]:
    target_df = target_df.sample(CFG["sample_num"]).reset_index(drop=True)
elif CFG["sample_num"] != -1:
    target_df = target_df.sample(CFG["sample_num"]).reset_index(drop=True)

print(len(target_df))
print(len(target_df["game_play"].unique()))
display(target_df["contact"].value_counts())

1049999
240


0    934533
1    115466
Name: contact, dtype: int64

In [20]:
target_df[CFG["features"]]

,x_position_1,y_position_1,x_position_2,y_position_2,speed_1,distance_1,direction_1,orientation_1,acceleration_1,sa_1,speed_2,distance_2,direction_2,orientation_2,acceleration_2,sa_2,players_dis
0,40.33,25.28,39.53,23.63,0.52,0.06,141.08,100.37,0.59,0.58,1.41,0.13,241.73,128.82,2.45,2.43,1.833712
1,40.33,25.28,40.11,26.73,0.52,0.06,141.08,100.37,0.59,0.58,0.99,0.09,163.38,90.69,1.68,1.64,1.466595
2,40.33,25.28,40.75,23.43,0.52,0.06,141.08,100.37,0.59,0.58,0.86,0.08,183.83,94.13,1.55,1.54,1.897077
3,42.00,22.85,40.75,23.43,0.68,0.08,234.17,282.07,0.81,0.81,0.86,0.08,183.83,94.13,1.55,1.54,1.378006
4,39.59,17.07,40.34,18.08,0.53,0.05,134.84,84.73,1.43,1.42,1.10,0.10,148.93,92.39,2.03,2.03,1.258014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049994,51.73,34.36,0.00,0.00,4.28,0.45,74.35,63.15,1.96,-1.96,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
1049995,62.95,42.49,0.00,0.00,0.67,0.07,252.10,255.20,0.14,-0.14,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
1049996,59.75,37.06,0.00,0.00,1.42,0.15,142.02,122.18,0.92,-0.11,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
1049997,58.80,40.24,0.00,0.00,1.50,0.15,206.60,137.36,0.89,-0.89,0.00,0.00,0.00,0.00,0.00,0.00,0.000000


# Augmentation

In [21]:
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.3, 0.3), p=0.5),
])

# Dataset

In [22]:
class NFLDataset(Dataset):
    def __init__(self, target_df, transform=None):
        self.target_df = target_df
        self.transform = transform

    def __len__(self):
        return len(self.target_df)

    def __getitem__(self, idx):
        target_info = self.target_df.iloc[idx]
        target = target_info.contact
        # read frame image
        game_play = target_info.game_play
        frame = target_info.frame
        view = target_info.EorS
        is_End = int(view=="Endzone")
        contact_id = target_info.contact_id
        contact_fileid = f"{contact_id}_{view}.jpg"
        contact_filename = os.path.join(CFG["CONTACT_IMG_DIR"], contact_fileid)
        img = cv2.imread(contact_filename)
        if img is None:
            img = np.zeros((224, 224, 3))
            img = np.transpose(img, (2, 0, 1)).astype(np.float)
            img = torch.tensor(img, dtype=torch.float)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if self.transform:
                img = self.transform(image=img)["image"]
            img = img / 255. # convert to 0-1
            img = np.transpose(img, (2, 0, 1)).astype(np.float32)
            img = torch.tensor(img, dtype=torch.float)
        target = torch.tensor(target, dtype=torch.float)

        return img, target, is_End

# Model

In [23]:
class NFLNet(nn.Module):
    def __init__(
        self,
        model_name = CFG["model_name"],
        out_features = CFG["num_img_feature"],
        inp_channels= CFG["inp_channels"],
        pretrained = CFG["pretrained"]
    ):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels, num_classes=out_features)
        self.fc = nn.Linear(out_features, 1)

    def forward(self, img):
        img_emb = self.model(img)
        output = self.fc(img_emb)
        return output, img_emb

# train fn

In [24]:
def train_fn(train_loader, model, criterion, epoch ,optimizer, scheduler):
    model.train()
    batch_time = AverageMeter()
    losses = AverageMeter()
    start, end = time.time(), time.time()
    for batch_idx, (images, targets, _) in enumerate(train_loader):
        images = images.to(device, non_blocking = True).float()
        targets = targets.to(device, non_blocking = True).float().view(-1, 1)      
        preds, _ = model(images)
        
        loss = criterion(preds, targets)
        losses.update(loss.item(), CFG["batch_size"]) 
        targets = targets.detach().cpu().numpy().ravel().tolist()
        preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()

        loss.backward() # パラメータの勾配を計算
        optimizer.step() # モデル更新
        optimizer.zero_grad() # 勾配の初期化
                
        batch_time.update(time.time() - end)
        end = time.time()
        if batch_idx % CFG["print_freq"] == 0 or batch_idx == (len(train_loader)-1):
            print('\t Epoch: [{0}][{1}/{2}] '
                    'Elapsed {remain:s} '
                    'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                    .format(
                        epoch, batch_idx, len(train_loader), batch_time=batch_time, loss=losses,
                        remain=timeSince(start, float(batch_idx+1)/len(train_loader)),
            ))
        del preds, images, targets
    gc.collect()
    torch.cuda.empty_cache()
    return losses.avg

# valid fn

In [25]:
def valid_fn(model, valid_loader, criterion):
    model.eval()# モデルを検証モードに設定
    test_targets = []
    test_preds = []
    img_embs = []

    batch_time = AverageMeter()
    losses = AverageMeter()
    start, end = time.time(), time.time()
    view_list = []
    for batch_idx, (images, targets, is_End) in enumerate(valid_loader):
        images = images.to(device, non_blocking = True).float()
        targets = targets.to(device, non_blocking = True).float().view(-1, 1)
        with torch.no_grad():
            preds, img_emb = model(images)
            loss = criterion(preds, targets)
        losses.update(loss.item(), CFG["batch_size"])
        batch_time.update(time.time() - end)

        img_emb = img_emb.detach().cpu().numpy()
        img_embs.extend(img_emb)

        targets = targets.detach().cpu().numpy().ravel().tolist()
        preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()

        test_preds.extend(preds)
        test_targets.extend(targets)
        view_list.extend(is_End.numpy().tolist())
        # score = matthews_corrcoef(preds, targets)
        if batch_idx % CFG["print_freq"] == 0 or batch_idx == (len(valid_loader)-1):
            print('\t EVAL: [{0}/{1}] '
                'Elapsed {remain:s} '
                'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                .format(
                    batch_idx, len(valid_loader), batch_time=batch_time, loss=losses,
                    remain=timeSince(start, float(batch_idx+1)/len(valid_loader)),
                ))
        del preds, images, targets
        gc.collect()
        torch.cuda.empty_cache()
    test_preds = np.array(test_preds)
    test_targets = np.array(test_targets)
    return test_targets, test_preds, img_embs, view_list, losses.avg

# Train loop

In [26]:
def training_loop(target_df):
    # set model & learning fn
    oof_df = pd.DataFrame()
    kf = GroupKFold(n_splits=CFG["n_folds"])
    for fold, (idx_train, idx_valid) in enumerate(kf.split(target_df, target_df["contact_id"], target_df["game_play"])):
        print("---")
        print(f"fold {fold} start training...")
        model = NFLNet()
        model = model.to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"], amsgrad=False)
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG["T_max"], eta_min=CFG["min_lr"], last_epoch=-1)

        if not fold in CFG["train_folds"]:
            print(f"fold{fold} is skip")
            continue
        # separate train/valid data 
        train_df = target_df.iloc[idx_train]
        valid_df = target_df.iloc[idx_valid]
        print("train target contact")
        print(train_df["contact"].value_counts())
        print("valid target contact")
        print(valid_df["contact"].value_counts())
        # separate train/valid data 
        train_dataset = NFLDataset(train_df, train_transform)
        # valid_dataset = NFLDataset(valid_df, valid_transform)
        valid_dataset = NFLDataset(valid_df,)
        train_loader = DataLoader(train_dataset,batch_size=CFG["batch_size"], shuffle = True,
                                    num_workers = CFG["num_workers"], pin_memory = True)
        valid_loader = DataLoader(valid_dataset,batch_size=CFG["batch_size"], shuffle = False,
                                    num_workers = CFG["num_workers"], pin_memory = True)

        # training
        best_score = -np.inf
        best_auc = -np.inf
        start_time, end = time.time(), time.time()
        for epoch in range(1, CFG["n_epoch"] + 1):
            print(f'\t === epoch: {epoch}: training ===')
            train_loss_avg = train_fn(train_loader, model, criterion, epoch ,optimizer, scheduler)
            valid_targets, valid_preds, valid_embs, view_list, valid_loss_avg = valid_fn(model, valid_loader, criterion)

            valid_score = -np.inf
            valid_threshold = 0
            tn_best, fp_best, fn_best, tp_best = 0, 0, 0, 0
            for idx in range(1, 10, 1):
                thr = idx*0.1
                valid_targets = (np.array(valid_targets) > thr).astype(np.int32)
                valid_binary_preds = (np.array(valid_preds) > thr).astype(np.int32)
                score_tmp = matthews_corrcoef(valid_targets, valid_binary_preds)
                cm = confusion_matrix(valid_targets, valid_binary_preds)
                tn, fp, fn, tp = cm.flatten()
                if score_tmp > valid_score:
                    valid_score = score_tmp 
                    valid_threshold = thr
                    tn_best, fp_best, fn_best, tp_best = tn, fp, fn, tp
            elapsed = (time.time() - start_time)/60
            auc_score = roc_auc_score(valid_targets, valid_preds)
            print(f'\t epoch:{epoch}, avg train loss:{train_loss_avg:.4f}, avg valid loss:{valid_loss_avg:.4f}')
            print(f'\t score:{valid_score:.4f}(th={valid_threshold}) AUC = {auc_score:.4f}=> time:{elapsed:.2f} min')
            scheduler.step()
            # validationスコアがbestを更新したらモデルを保存する
            if valid_score > best_score:
                best_score = valid_score
                model_name = CFG["model_name"]
                torch.save(model.state_dict(), f'{CFG["MODEL_DIR"]}/{model_name}_fold{fold}.pth')
                print(f'\t Epoch {epoch} - Save Best Score: {best_score:.4f}. Model is saved.')
                contact_id = valid_df["contact_id"].values
                _oof_df = pd.DataFrame({
                    "contact_id" : contact_id,
                    "pred" : valid_preds,
                    "contact" : valid_targets,
                    "fold" : fold,
                    "is_End" : view_list,
                })
                img_emb_colname = [f"img_emb_{idx}" for idx in range(CFG["num_img_feature"])]
                img_emb_df = pd.DataFrame(valid_embs, columns=img_emb_colname)
                _oof_df = pd.concat([_oof_df, img_emb_df], axis=1)
            
            logging_metrics_epoch(fold, epoch, train_loss_avg, valid_loss_avg, valid_score, valid_threshold, tn_best, fp_best, fn_best, tp_best, auc_score)

        del train_loader, train_dataset, valid_loader, valid_dataset
        oof_df = pd.concat([oof_df, _oof_df], axis = 0)
        del _oof_df
        gc.collect()
        torch.cuda.empty_cache()
    return oof_df

In [27]:
if CFG["kaggle"]:
    oof_df = training_loop(target_df)
    wandb.finish()
else:
    with mlflow.start_run(experiment_id=experiment_id, run_name=CFG["EXP_NAME"]) as run:
        mlflow.log_dict(CFG, "configuration.yaml")
        mlflow.log_param("positive data num", len(target_df[target_df["contact"]==1]))
        mlflow.log_param("negative data num", len(target_df[target_df["contact"]==0]))
        oof_df = training_loop(target_df)

---
fold 0 start training...
train target contact
0    621992
1     77999
Name: contact, dtype: int64
valid target contact
0    312541
1     37467
Name: contact, dtype: int64
	 === epoch: 1: training ===
	 Epoch: [1][0/5469] Elapsed 0m 1s (remain 160m 4s) Loss: 1.0625(1.0625) 
	 Epoch: [1][1000/5469] Elapsed 7m 27s (remain 33m 15s) Loss: 0.2498(0.2465) 
	 Epoch: [1][2000/5469] Elapsed 14m 54s (remain 25m 50s) Loss: 0.2751(0.2254) 
	 Epoch: [1][3000/5469] Elapsed 22m 15s (remain 18m 18s) Loss: 0.1596(0.2152) 
	 Epoch: [1][4000/5469] Elapsed 29m 36s (remain 10m 51s) Loss: 0.1539(0.2093) 
	 Epoch: [1][5000/5469] Elapsed 36m 59s (remain 3m 27s) Loss: 0.1153(0.2043) 
	 Epoch: [1][5468/5469] Elapsed 40m 27s (remain 0m 0s) Loss: 0.1293(0.2024) 
	 EVAL: [0/2735] Elapsed 0m 0s (remain 44m 39s) Loss: 0.0525(0.0525) 
	 EVAL: [1000/2735] Elapsed 7m 20s (remain 12m 43s) Loss: 0.2497(0.2047) 
	 EVAL: [2000/2735] Elapsed 14m 28s (remain 5m 18s) Loss: 0.0152(0.2026) 
	 EVAL: [2734/2735] Elapsed 19m 45

# Save oof_df

In [28]:
display(oof_df)
if CFG["kaggle"]:
    oof_filename = os.path.join(CFG["OUTPUT_DIR"], "oof_df.csv")
    oof_df.to_csv(oof_filename, index=False)
else:
    oof_filename = os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"], "oof_df.csv")
    oof_df.to_csv(oof_filename, index=False)

,contact_id,pred,contact,fold,is_End,img_emb_0,img_emb_1,img_emb_2,img_emb_3,img_emb_4
0,58168_003392_0_38590_44822,0.000012,0,0,1,10.889146,-10.556498,3.099932,7.830846,9.278116
1,58168_003392_0_38590_39947,0.000256,0,0,1,6.427232,-8.590977,2.088487,3.197328,8.112932
2,58168_003392_0_38590_42565,0.000291,0,0,1,7.171048,-8.056860,2.414712,4.299431,6.933124
3,58168_003392_0_41944_42565,0.097480,0,0,1,2.284749,-1.047679,0.416023,0.065301,3.101881
4,58168_003392_0_37211_46445,0.009431,0,0,1,5.025501,-3.066904,1.134434,0.261845,6.074508
...,...,...,...,...,...,...,...,...,...,...
349972,58579_003527_80_47848_G,0.004093,0,2,0,6.409846,3.144198,-1.802670,-11.054835,3.547827
349973,58579_003527_80_38551_G,0.000027,0,2,0,5.817370,1.663301,6.231243,-11.347146,12.625340
349974,58579_003527_80_52416_G,0.000010,0,2,0,5.598929,9.255042,7.504594,-10.636785,13.018196
349975,58579_003527_80_47819_G,0.000052,0,2,0,6.901435,3.672146,4.153616,-11.483387,10.734515


In [29]:
for idx in range(1, 10, 1):
    thr = idx*0.1
    valid_targets = (np.array(oof_df["contact"]) > thr).astype(np.int32)
    valid_binary_preds = (np.array(oof_df["pred"]) > thr).astype(np.int32)
    score = matthews_corrcoef(valid_targets, valid_binary_preds)
    print(f"threshold={thr:.3f}, score={score:.5f}")

threshold=0.100, score=0.51316
threshold=0.200, score=0.55528
threshold=0.300, score=0.56317
threshold=0.400, score=0.55265
threshold=0.500, score=0.53089
threshold=0.600, score=0.49367
threshold=0.700, score=0.44083
threshold=0.800, score=0.36671
threshold=0.900, score=0.25609


In [30]:
alldata_df = pd.read_csv("/workspace/input/train_labels.csv")
alldata_df = alldata_df.merge(oof_df[["contact_id", "pred"]], on="contact_id", how="left")

In [31]:
for idx in range(1, 10, 1):
    thr = idx*0.1
    valid_targets = (np.array(alldata_df["contact"]) > thr).astype(np.int32)
    valid_binary_preds = (np.array(alldata_df["pred"]) > thr).astype(np.int32)
    score = matthews_corrcoef(valid_targets, valid_binary_preds)
    print(f"threshold={thr:.3f}, score={score:.5f}")

threshold=0.100, score=0.56485
threshold=0.200, score=0.59606
threshold=0.300, score=0.59848
threshold=0.400, score=0.58425
threshold=0.500, score=0.55907
threshold=0.600, score=0.51874
threshold=0.700, score=0.46242
threshold=0.800, score=0.38422
threshold=0.900, score=0.26808
